In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
from scipy.spatial.distance import mahalanobis as mah_dist # u, v, cov_mat
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))


def po_linear_model(graph, alpha=1, beta=1, sigma=0.1, hete=0.5, gamma=2):    
    hete = 0.5
    for i in graph.nodes:
        # graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() + hete * graph.degree[i]/avg_deg + gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i]  
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i]  

def po_multiplicative_model(graph, alpha=1, sigma=0.1, delta=1, gamma=2): 
    for i in graph.nodes:
        graph.nodes[i]["y"] = ( (alpha + sigma * np.random.normal()) * graph.degree[i]/avg_deg )  *  (1 + delta * graph.nodes[i]["z"] + gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]]) / len(graph[i]) )




DGL backend not selected or invalid.  Assuming PyTorch for now.
/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["n_cl"] = len(node_to_connected_clusters[i])
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)
        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [6]:
init_seed = 649 # interupted
GATE_hat_list = torch.load("Result/Multiplicative_incre_2step_CR_unit.pkl")

In [ ]:
# unit-level cr

# GATE_hat_list = np.zeros((repeat_num, len(ramps)))

     
# for seed in tqdm(range(repeat_num)):
for seed in tqdm(range(init_seed, repeat_num)):
    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_nodes,))   
    
    for num_step in range(len(ramps)):
        
        torch.manual_seed(2)
        p_list = [0, ramps[num_step]]
    
        print(f"Seed: {seed}")
        
        g_feat_list = []
        y_list = []
        
        for p in p_list:
            
            # tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]
            
            # tr_units = reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters])  
            z = (rollout_index < np.quantile(rollout_index, p))
            
            nx.set_node_attributes(g, 0, "z")
            
            nx.set_node_attributes(g, {unit:1 for unit in range(num_nodes) if z[unit]}, "z")
            # po_linear_model(g, gamma = 1)
            po_multiplicative_model(g, gamma = 1)
            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()
        # optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
    
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
        
    
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    torch.save(GATE_hat_list, "Result/Multiplicative_incre_2step_CR_unit.pkl")


  0%|          | 0/351 [00:00<?, ?it/s]

Seed: 649


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.00, Loss: 4.5442
Epoch 0, Treat Prop: 0.02, Loss: 0.6770
Epoch 50, Treat Prop: 0.00, Loss: 0.0226
Epoch 50, Treat Prop: 0.02, Loss: 0.0611
Epoch 100, Treat Prop: 0.00, Loss: 0.0208
Epoch 100, Treat Prop: 0.02, Loss: 0.0574
Epoch 150, Treat Prop: 0.00, Loss: 0.0209
Epoch 150, Treat Prop: 0.02, Loss: 0.0539
Epoch 200, Treat Prop: 0.00, Loss: 0.0207
Epoch 200, Treat Prop: 0.02, Loss: 0.0514
Epoch 250, Treat Prop: 0.00, Loss: 0.0207
Epoch 250, Treat Prop: 0.02, Loss: 0.0491
Epoch 300, Treat Prop: 0.00, Loss: 0.0207
Epoch 300, Treat Prop: 0.02, Loss: 0.0468
Epoch 350, Treat Prop: 0.00, Loss: 0.0207
Epoch 350, Treat Prop: 0.02, Loss: 0.0444
0.04201940447092056
Seed: 649
Epoch 0, Treat Prop: 0.00, Loss: 4.5410
Epoch 0, Treat Prop: 0.05, Loss: 0.6647
Epoch 50, Treat Prop: 0.00, Loss: 0.0290
Epoch 50, Treat Prop: 0.05, Loss: 0.1147
Epoch 100, Treat Prop: 0.00, Loss: 0.0265
Epoch 100, Treat Prop: 0.05, Loss: 0.1072
Epoch 150, Treat Prop: 0.00, Loss: 0.0259
Epoch 150, Treat

  0%|          | 1/351 [00:21<2:07:30, 21.86s/it]

...

0.22078290581703186
Seed: 997
Epoch 0, Treat Prop: 0.00, Loss: 4.5615
Epoch 0, Treat Prop: 0.02, Loss: 0.6647
Epoch 50, Treat Prop: 0.00, Loss: 0.0236
Epoch 50, Treat Prop: 0.02, Loss: 0.0647
Epoch 100, Treat Prop: 0.00, Loss: 0.0223
Epoch 100, Treat Prop: 0.02, Loss: 0.0602
Epoch 150, Treat Prop: 0.00, Loss: 0.0218
Epoch 150, Treat Prop: 0.02, Loss: 0.0573
Epoch 200, Treat Prop: 0.00, Loss: 0.0218
Epoch 200, Treat Prop: 0.02, Loss: 0.0547
Epoch 250, Treat Prop: 0.00, Loss: 0.0218
Epoch 250, Treat Prop: 0.02, Loss: 0.0522
Epoch 300, Treat Prop: 0.00, Loss: 0.0218
Epoch 300, Treat Prop: 0.02, Loss: 0.0496
Epoch 350, Treat Prop: 0.00, Loss: 0.0219
Epoch 350, Treat Prop: 0.02, Loss: 0.0470
0.04432243853807449
Seed: 997
Epoch 0, Treat Prop: 0.00, Loss: 4.5504
Epoch 0, Treat Prop: 0.05, Loss: 0.6605
Epoch 50, Treat Prop: 0.00, Loss: 0.0304
Epoch 50, Treat Prop: 0.05, Loss: 0.1355
Epoch 100, Treat Prop: 0.00, Loss: 0.0285
Epoch 100, Treat Prop: 0.05, Loss: 0.1282
Epoch 150, Treat Prop: 0.00,

 99%|█████████▉| 349/351 [1:57:35<00:39, 19.88s/it]

0.16197724640369415
Seed: 998
Epoch 0, Treat Prop: 0.00, Loss: 4.5516
Epoch 0, Treat Prop: 0.02, Loss: 0.6440
Epoch 50, Treat Prop: 0.00, Loss: 0.0228
Epoch 50, Treat Prop: 0.02, Loss: 0.0606
Epoch 100, Treat Prop: 0.00, Loss: 0.0213
Epoch 100, Treat Prop: 0.02, Loss: 0.0564
Epoch 150, Treat Prop: 0.00, Loss: 0.0208
Epoch 150, Treat Prop: 0.02, Loss: 0.0536
Epoch 200, Treat Prop: 0.00, Loss: 0.0207
Epoch 200, Treat Prop: 0.02, Loss: 0.0512
Epoch 250, Treat Prop: 0.00, Loss: 0.0207
Epoch 250, Treat Prop: 0.02, Loss: 0.0488
Epoch 300, Treat Prop: 0.00, Loss: 0.0207
Epoch 300, Treat Prop: 0.02, Loss: 0.0465
Epoch 350, Treat Prop: 0.00, Loss: 0.0207
Epoch 350, Treat Prop: 0.02, Loss: 0.0443
0.04215305671095848
Seed: 998
Epoch 0, Treat Prop: 0.00, Loss: 4.5398
Epoch 0, Treat Prop: 0.05, Loss: 0.6612
Epoch 50, Treat Prop: 0.00, Loss: 0.0289
Epoch 50, Treat Prop: 0.05, Loss: 0.1141
Epoch 100, Treat Prop: 0.00, Loss: 0.0267
Epoch 100, Treat Prop: 0.05, Loss: 0.1067
Epoch 150, Treat Prop: 0.00,

100%|█████████▉| 350/351 [1:57:55<00:19, 19.89s/it]

0.17876935005187988
Seed: 999
Epoch 0, Treat Prop: 0.00, Loss: 4.5572
Epoch 0, Treat Prop: 0.02, Loss: 0.6612
Epoch 50, Treat Prop: 0.00, Loss: 0.0226
Epoch 50, Treat Prop: 0.02, Loss: 0.0720
Epoch 100, Treat Prop: 0.00, Loss: 0.0211
Epoch 100, Treat Prop: 0.02, Loss: 0.0676
Epoch 150, Treat Prop: 0.00, Loss: 0.0206
Epoch 150, Treat Prop: 0.02, Loss: 0.0644
Epoch 200, Treat Prop: 0.00, Loss: 0.0205
Epoch 200, Treat Prop: 0.02, Loss: 0.0614
Epoch 250, Treat Prop: 0.00, Loss: 0.0205
Epoch 250, Treat Prop: 0.02, Loss: 0.0585
Epoch 300, Treat Prop: 0.00, Loss: 0.0206
Epoch 300, Treat Prop: 0.02, Loss: 0.0553
Epoch 350, Treat Prop: 0.00, Loss: 0.0206
Epoch 350, Treat Prop: 0.02, Loss: 0.0520
0.04880334436893463
Seed: 999
Epoch 0, Treat Prop: 0.00, Loss: 4.5618
Epoch 0, Treat Prop: 0.05, Loss: 0.6736
Epoch 50, Treat Prop: 0.00, Loss: 0.0301
Epoch 50, Treat Prop: 0.05, Loss: 0.1317
Epoch 100, Treat Prop: 0.00, Loss: 0.0285
Epoch 100, Treat Prop: 0.05, Loss: 0.1242
Epoch 150, Treat Prop: 0.00,